In [51]:
!pip3 install datawig

Processing c:\users\rupal\appdata\local\pip\cache\wheels\9b\a2\80\f3c44c1e8357a5ef3e85a0fd7a323511266ec42762d91e1025\datawig-0.1.12-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4a/bd/eee1157fc2d8514970b345d69cb9975dcd1e42cd7e61146ed841f6e68309/typing-3.6.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f0/ac/92c3d2f0b627efbd1a7b2156faa697f9c2bbd7b0fe83ba8a9d36f982156f/pandas-0.25.3-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/d0/19/6d56792b95445fc9b818e8d8eff3d5fff1221c40fe06a096c9587f2cb1d3/mxnet-1.4.0-py2.py3-none-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/53/39/4ab213673844e0c004bed8a0781a0721a3f6bb23eb8854ee75c236428892/graphviz-0.8.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/c

ERROR: py2neo 4.1.0 has requirement prompt-toolkit==1.0.15, but you'll have prompt-toolkit 2.0.7 which is incompatible.
ERROR: mxnet 1.4.0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'd:\\users\\rupal\\anaconda3\\lib\\site-packages\\~andas\\io\\msgpack\\_packer.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [21]:
import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport

from jenga.tasks.income import IncomeEstimationTask
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

import datawig

#### Data

In [74]:
dat = IncomeEstimationTask(seed=10)

In [75]:
train_dat = dat.train_data
train_lab = dat.train_labels

In [62]:
print(type(train_dat))
print(type(train_lab))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [78]:
train_dat.head()

,workclass,occupation,marital_status,education,hours_per_week,age
190,Private,Machine-op-inspct,Divorced,Some-college,28,38
22645,Private,Adm-clerical,Married-civ-spouse,Some-college,40,27
4802,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,25
16052,Private,Transport-moving,Married-civ-spouse,HS-grad,50,51
11580,NaN,NaN,Married-civ-spouse,HS-grad,15,71


In [25]:
profile = ProfileReport(train_dat, title="Training Data Profiling Report")

In [26]:
profile

Workclass and Occupation are the only columns with missing values in the training data

In [7]:
train_lab[0:5]

array([ True,  True, False, False, False])

In [8]:
test_dat = dat.test_data

#### Simple Model

In [9]:
# fit a baseline model
model = dat.fit_baseline_model(train_dat, train_lab)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   12.4s finished


In [10]:
# get the prediction probability estimates on the test set
pred_prob = model.predict_proba(test_dat)
dat.score_on_test_data(pred_prob)

0.8819507431977972

#### Test on corrupted data

In [11]:
# corrupt the test data
missing_values_corruption = MissingValues(column="age", fraction=0.45, missingness="MAR", na_value=999)

corrupt_test_dat = missing_values_corruption.transform(test_dat)
corrupt_test_dat

,workclass,occupation,marital_status,education,hours_per_week,age
14160,Private,Adm-clerical,Divorced,Some-college,38,27
27048,State-gov,Exec-managerial,Married-civ-spouse,HS-grad,40,999
28868,Private,Exec-managerial,Married-civ-spouse,Bachelors,55,999
5667,Private,Machine-op-inspct,Never-married,Bachelors,40,30
7827,Self-emp-not-inc,Craft-repair,Divorced,Some-college,50,999
...,...,...,...,...,...,...
1338,Private,Tech-support,Divorced,Bachelors,16,71
24534,Local-gov,Prof-specialty,Married-civ-spouse,Some-college,40,999
18080,Private,Prof-specialty,Married-civ-spouse,Prof-school,48,999
10354,Private,Adm-clerical,Never-married,Bachelors,40,27


In [12]:
# get the prediction probability estimates on the corrupted test set
pred_prob_currupt = model.predict_proba(corrupt_test_dat)
dat.score_on_test_data(pred_prob_currupt)

0.8446729355237496

#### Evaluate the impact of corruptions on the baseline model
using the evaluator from Jenga

In [13]:
# corruption impact evaluator
evaluator = CorruptionImpactEvaluator(dat)

num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, missing_values_corruption)

0/10 (0.046875)


In [14]:
results

In [15]:
print(f"""
Score (AUC) on
    data: {results[0].baseline_score}
    corrupted data: {np.mean(results[0].corrupted_scores)}
""")


Score (AUC) on
    data: 0.8819507431977972
    corrupted data: 0.7260489147052982



In [ ]:
### analyse the result
### see how the evaluator works
### analyse the difference in scores using evaluator and predict_proba

#### Try imputing the columns with missing values using Datawig

In [29]:
## training imputation models for the columns with null values: workclass and occupation
## both are categorical attributes

# workclass
workclass_imputer = datawig.SimpleImputer(
    input_columns=['occupation', 'marital_status', 'education'],
    output_column='workclass',
    output_path='imputer_model_workclass')

workclass_imputer.fit(train_dat)

2020-06-01 16:17:26,798 [INFO]  CategoricalEncoder for column workclass                                found only 9 occurrences of value Without-pay
2020-06-01 16:17:26,798 [INFO]  CategoricalEncoder for column workclass                                found only 5 occurrences of value Never-worked
2020-06-01 16:17:38,329 [INFO]  
========== start: fit model
2020-06-01 16:17:38,345 [WARNING]  Already bound, ignoring bind()
2020-06-01 16:17:58,633 [INFO]  Epoch[0] Batch [0-692]	Speed: 548.01 samples/sec	cross-entropy=0.876242	workclass-accuracy=0.732864
2020-06-01 16:18:17,632 [INFO]  Epoch[0] Train-cross-entropy=0.843446
2020-06-01 16:18:17,632 [INFO]  Epoch[0] Train-workclass-accuracy=0.738665
2020-06-01 16:18:17,632 [INFO]  Epoch[0] Time cost=39.280
2020-06-01 16:18:17,663 [INFO]  Saved checkpoint to "workclass_imputer_model\model-0000.params"
2020-06-01 16:18:20,601 [INFO]  Epoch[0] Validation-cross-entropy=0.809529
2020-06-01 16:18:20,608 [INFO]  Epoch[0] Validation-workclass-accura

In [30]:
# occupation
occupation_imputer = datawig.SimpleImputer(
    input_columns=['workclass', 'marital_status', 'education'],
    output_column='occupation',
    output_path='imputer_model_occupation')

occupation_imputer.fit(train_dat)

2020-06-01 16:28:24,079 [INFO]  CategoricalEncoder for column occupation                                found only 6 occurrences of value Armed-Forces
2020-06-01 16:28:35,068 [WARNING]  Test set does not contain any ocurrences of values [Armed-Forces] in column [occupation], consider using a more representative test set.
2020-06-01 16:28:35,084 [INFO]  
========== start: fit model
2020-06-01 16:28:35,084 [WARNING]  Already bound, ignoring bind()
2020-06-01 16:28:59,197 [INFO]  Epoch[0] Batch [0-692]	Speed: 461.44 samples/sec	cross-entropy=1.991844	occupation-accuracy=0.276245
2020-06-01 16:29:22,500 [INFO]  Epoch[0] Train-cross-entropy=1.946666
2020-06-01 16:29:22,500 [INFO]  Epoch[0] Train-occupation-accuracy=0.285495
2020-06-01 16:29:22,500 [INFO]  Epoch[0] Time cost=47.410
2020-06-01 16:29:22,553 [INFO]  Saved checkpoint to "occupation_imputer_model\model-0000.params"
2020-06-01 16:29:25,492 [INFO]  Epoch[0] Validation-cross-entropy=1.884101
2020-06-01 16:29:25,492 [INFO]  Epoch[0] 

In [77]:
## impute the columns in the training data
imputed = workclass_imputer.predict(train_dat)

In [90]:
imputed_copy = imputed.copy(deep=True)
imputed_copy

,workclass,occupation,marital_status,education,hours_per_week,age,workclass_imputed,workclass_imputed_proba
190,Private,Machine-op-inspct,Divorced,Some-college,28,38,Private,0.956437
22645,Private,Adm-clerical,Married-civ-spouse,Some-college,40,27,Private,0.667604
4802,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,25,Private,0.950972
16052,Private,Transport-moving,Married-civ-spouse,HS-grad,50,51,Private,0.804180
11580,NaN,NaN,Married-civ-spouse,HS-grad,15,71,Never-worked,0.979607
...,...,...,...,...,...,...,...,...
28017,Private,Sales,Never-married,Some-college,30,27,Private,0.896090
17728,Private,Craft-repair,Divorced,Assoc-acdm,45,35,Private,0.788922
29199,Private,Sales,Never-married,Some-college,40,19,Private,0.896090
7293,Private,Other-service,Never-married,11th,40,34,Private,0.941356


In [93]:
# checking cases like index 11580 
imputed_copy.loc[imputed_copy["workclass"].isnull()& imputed_copy["occupation"].isnull()]

,workclass,occupation,marital_status,education,hours_per_week,age,workclass_imputed,workclass_imputed_proba
11580,NaN,NaN,Married-civ-spouse,HS-grad,15,71,Never-worked,0.979607
32541,NaN,NaN,Separated,HS-grad,32,41,Never-worked,0.662470
1823,NaN,NaN,Never-married,Some-college,45,19,Never-worked,0.986100
20038,NaN,NaN,Married-civ-spouse,HS-grad,40,49,Never-worked,0.979607
3670,NaN,NaN,Divorced,HS-grad,75,31,Never-worked,0.988336
...,...,...,...,...,...,...,...,...
22841,NaN,NaN,Never-married,Some-college,40,26,Never-worked,0.986100
19986,NaN,NaN,Never-married,Some-college,22,19,Never-worked,0.986100
31390,NaN,NaN,Married-civ-spouse,Masters,2,61,Never-worked,0.427696
17790,NaN,NaN,Married-civ-spouse,Bachelors,16,44,Private,0.498835


In [94]:
imputed.loc[imputed["workclass"].isnull(), "workclass"] = imputed["workclass" + '_imputed']

In [95]:
imputed.loc[imputed["workclass"].isnull()]

,workclass,occupation,marital_status,education,hours_per_week,age,workclass_imputed,workclass_imputed_proba


In [98]:
imputed

,workclass,occupation,marital_status,education,hours_per_week,age,workclass_imputed,workclass_imputed_proba
190,Private,Machine-op-inspct,Divorced,Some-college,28,38,Private,0.956437
22645,Private,Adm-clerical,Married-civ-spouse,Some-college,40,27,Private,0.667604
4802,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,25,Private,0.950972
16052,Private,Transport-moving,Married-civ-spouse,HS-grad,50,51,Private,0.804180
11580,Never-worked,NaN,Married-civ-spouse,HS-grad,15,71,Never-worked,0.979607
...,...,...,...,...,...,...,...,...
28017,Private,Sales,Never-married,Some-college,30,27,Private,0.896090
17728,Private,Craft-repair,Divorced,Assoc-acdm,45,35,Private,0.788922
29199,Private,Sales,Never-married,Some-college,40,19,Private,0.896090
7293,Private,Other-service,Never-married,11th,40,34,Private,0.941356


Notice update for the index 11580

In [109]:
# maintaining the structure of the imputed df the same as the training df
imputed[train_dat.columns]

,workclass,occupation,marital_status,education,hours_per_week,age
190,Private,Machine-op-inspct,Divorced,Some-college,28,38
22645,Private,Adm-clerical,Married-civ-spouse,Some-college,40,27
4802,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,25
16052,Private,Transport-moving,Married-civ-spouse,HS-grad,50,51
11580,Never-worked,NaN,Married-civ-spouse,HS-grad,15,71
...,...,...,...,...,...,...
28017,Private,Sales,Never-married,Some-college,30,27
17728,Private,Craft-repair,Divorced,Assoc-acdm,45,35
29199,Private,Sales,Never-married,Some-college,40,19
7293,Private,Other-service,Never-married,11th,40,34


In [ ]:
## impute the columns in the training data
def impute_dat(data, imputers):
    imputed = data.copy(deep=True)
    for imputer in imputers:
        imputed = imputer.predict(imputed, inplace=True)
    
    return imputed

In [110]:
## impute the columns in the training data
def impute_dat(data, columns, imputers):
    imputed = data.copy(deep=True)
    for imputer in imputers:
        imputed = imputer.predict(imputed, inplace=True)
    
    for column in columns:
        imputed.loc[imputed[column].isnull(), column] = imputed[column + '_imputed']
    
    return imputed[data.columns]

In [113]:
## impute the training data for missing values in "workclass" and "occupation"
train_dat_imputed = impute_dat(train_dat, ["workclass", "occupation"],[workclass_imputer, occupation_imputer])

In [114]:
train_dat_imputed

,workclass,occupation,marital_status,education,hours_per_week,age
190,Private,Machine-op-inspct,Divorced,Some-college,28,38
22645,Private,Adm-clerical,Married-civ-spouse,Some-college,40,27
4802,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,25
16052,Private,Transport-moving,Married-civ-spouse,HS-grad,50,51
11580,Never-worked,Craft-repair,Married-civ-spouse,HS-grad,15,71
...,...,...,...,...,...,...
28017,Private,Sales,Never-married,Some-college,30,27
17728,Private,Craft-repair,Divorced,Assoc-acdm,45,35
29199,Private,Sales,Never-married,Some-college,40,19
7293,Private,Other-service,Never-married,11th,40,34


#### Fit the baseline model on the imputed data
Need to impute the test data as well? 

In [116]:
model_imputed = dat.fit_baseline_model(train_dat_imputed, train_lab)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   14.1s finished


In [117]:
# get the prediction probability estimates on the test set
pred_prob_imputed = model_imputed.predict_proba(test_dat)
dat.score_on_test_data(pred_prob_imputed)

0.49779968994059076

In [118]:
# get the prediction probability estimates on the corrupted test set - makes sense?
pred_prob_imputed_corrupt = model_imputed.predict_proba(corrupt_test_dat)
dat.score_on_test_data(pred_prob_imputed_corrupt)

0.49697516604470343

#### Glue code (decorators) to be able to apply imputers
(taken from the sikm-submi.... notebook)

In [31]:
class ModelDecorator:
    def __init__(self, model, target_column, imputers):
        self.model = model
        self.imputers = imputers
        
    def predict_proba(self, data):
        imputed = data
        for imputer in self.imputers:
            imputed = imputer.transform(imputed)
            
        return self.model.predict_proba(imputed)

In [32]:
## we already have a baseline model, here I'm creating a model with datawig imputers
datawig_model = ModelDecorator(model, [workclass_imputer, occupation_imputer])

#### Evaluate the impact of corruptions on the model with datawig imputation

In [33]:
results_decorated = evaluator.evaluate(datawig_model, num_repetitions, missing_values_corruption)

AttributeError: 'SimpleImputer' object has no attribute 'transform'

#### Scenarios from the CleanML paper